# Vienna Land Use

The ..

In [ ]:
# 1. transformation of land use vector dataset to the 100m Vienna GRID:

# geometry:
import geopandas as gpd

# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp")
# 2 LOAD "land uise"
shp = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\landnutzung\REALNUT2020OGD_31256.shp")

# Perform spatial union
union_result = gpd.overlay(shp_100m_GRID, shp, how='union')

# Save the result to a new shapefile
union_result.to_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m.shp")

print ("done")




C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\shapely\set_operations.py:76: RuntimeWarning: divide by zero encountered in difference
  return lib.difference(a, b, **kwargs)
C:\Users\ManuelLöhnertz\AppData\Roaming\Python\Python310\site-packages\shapely\set_operations.py:76: RuntimeWarning: invalid value encountered in difference
  return lib.difference(a, b, **kwargs)
C:\Users\ManuelLöhnertz\AppData\Local\Temp\ipykernel_41516\3323111743.py:12: UserWarning: `keep_geom_type=True` in overlay resulted in 8 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  union_result = gpd.overlay(shp_100m_GRID, shp, how='union')


done


The following script calc. the area of land use class inside every 100x100m cell. Then the land use with the larrgest area will be selected and exported to  new vector grid file:

# 2) for every 100m cell calc area of 100x100m cover in percent:

In [66]:
import pandas as pd



## Land use table with FLY_indicator:


'''
NUTZUNG_CODE	NUTZUNG_LE	NUTZUNG_L0	NUTZUNG_L1	FLY_indiciator
1	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	locker bebautes Wohn(misch)gebiet	2
2	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	Wohn(misch)gebiet mittlerer Dichte	1
3	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	dichtes Wohn(misch)gebiet	1
4	Baulandnutzung	Wohn- u. Mischnutzung (Schwerpunkt Wohnen)	grossvolumiger, solidaerer Wohn(misch)bau	1
5	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Buero- und Verwaltungsviertel	1
6	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	solitaere Handelsstrukturen	1
7	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Geschaefts-, Kern- u. Mischgebiete	1
8	Baulandnutzung	Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)	Mischnutzung wenig dicht	1
9	Baulandnutzung	Industrie- und Gewerbenutzung	Industrie, prod. Gewerbe, Grosshandel inkl. Lager	0
10	Baulandnutzung	soziale Infrastruktur	Kultur, Freizeit, Messe	1
11	Baulandnutzung	soziale Infrastruktur	Gesundheit und Einsatzorg.	1
12	Baulandnutzung	soziale Infrastruktur	Bildung	1
13	Baulandnutzung	soziale Infrastruktur	Sport und Bad (Indoor)	0
14	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Militaerische Anlagen	1
15	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Klaeranlage, Deponie	2
16	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Energieversorgung u. Rundfunkanlagen	0
17	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Wasserversorgung	0
18	Baulandnutzung	Technische Infrastruktur/Kunstbauten/Sondernutzung	Transformationsflaeche, Baustelle, Materialgew.	0
19	Verkehr	Strassenraum	Strassenraum begruent	1
20	Verkehr	Strassenraum	Strassenraum unbegruent	0
21	Verkehr	weitere verkehrliche Nutzungen	Parkpluetze u. Parkhaeuser	0
22	Verkehr	weitere verkehrliche Nutzungen	Bahnhoefe und Bahnanlagen	0
23	Verkehr	weitere verkehrliche Nutzungen	Transport und Logistik inkl. Lager	0
24	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Park, Gruenanlage	1
25	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Sport und Bad (Outdoor), Camping	1
26	Gruenlandnutzung	Erholungs- u. Freizeiteinrichtungen	Friedhof	1
27	Gruenlandnutzung	Landwirtschaft	Acker	1
28	Gruenlandnutzung	Landwirtschaft	Weingarten	2
29	Gruenlandnutzung	Landwirtschaft	Gaertnerei, Obstplantagen	2
30	Gruenlandnutzung	Naturraum	Wald	1
31	Gruenlandnutzung	Naturraum	Wiese	1
32	Gruenlandnutzung	Gewaeser	Gewaeser inkl. Bachbett	0


'''


import pandas as pd

data2 = {
    'NUTZUNG_CODE': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
  

        'NUTZUNG_LE3': [  'locker bebautes Wohn(misch)gebiet', 
                     'Wohn(misch)gebiet mittlerer Dichte', 
                     'dichtes Wohn(misch)gebiet',
                     'grossvolumiger und solidaerer Wohn(misch)bau', 
                     'Buero- und Verwaltungsviertel', 
                     'solitaere Handelsstrukturen', 
                     'Geschaefts- u. Kern- u. Mischgebiete', 
                     'Mischnutzung wenig dicht', 
                     'Industrie prod. Gewerbe, Grosshandel inkl. Lager',
                     'Kultur und  Freizeit und  Messe', 
                     'Gesundheit und Einsatzorg', 
                     'Bildung', 
                     'Sport und Bad (Indoor)', 
                     'Militaerische Anlagen', 
                     'Klaeranlage und  Deponie', 
                     'Energieversorgung u. Rundfunkanlagen', 
                     'Wasserversorgung', 
                     'Transformationsflaeche  ,Baustelle, Materialgew.', 
                     'Strassenraum begruent', 
                     'Strassenraum unbegruent',
                     'Parkpluetze u. Parkhaeuser', 
                     'Bahnhoefe und Bahnanlagen', 
                     'Transport und Logistik inkl. Lager',
                     'Park und Gruenanlage', 
                     'Sport und Bad (Outdoor) Camping', 
                     'Friedhof', 
                     'Acker', 
                     'Weingarten',
                     'Gaertnerei und Obstplantagen', 
                     'Wald',
                     'Wiese', 
                     'Gewaeser inkl. Bachbett'],

                     'NUTZUNG_LE2': [

                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Wohn- u. Mischnutzung (Schwerpunkt Wohnen)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Geschaefts,- Kern- und Mischnutzung (Schwerpunkt betriebl. Taetigkeit)',
                                'Industrie- und Gewerbenutzung',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'soziale Infrastruktur',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Technische Infrastruktur/Kunstbauten/Sondernutzung',
                                'Strassenraum',
                                'Strassenraum',
                                'weitere verkehrliche Nutzungen',
                                'weitere verkehrliche Nutzungen',
                                'weitere verkehrliche Nutzungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Erholungs- u. Freizeiteinrichtungen',
                                'Landwirtschaft',
                                'Landwirtschaft',
                                'Landwirtschaft',
                                'Naturraum',
                                'Naturraum',
                                'Gewaeser'],



                       'NUTZUNG_LE1': [
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Baulandnutzung',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Verkehr',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung',
                        'Gruenlandnutzung'],
                 'FLY_indiciator': [
                        '2',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '1',
                        '0',
                        '1',
                        '1',
                        '1',
                        '0',
                        '1',
                        '2',
                        '0',
                        '0',
                        '0',
                        '1',
                        '0',
                        '0',
                        '0',
                        '0',
                        '1',
                        '1',
                        '1',
                        '1',
                        '2',
                        '2',
                        '1',
                        '1',
                        '0']

}

df_land_use_LUT = pd.DataFrame(data2)

(df_land_use_LUT)



,NUTZUNG_CODE,NUTZUNG_LE3,NUTZUNG_LE2,NUTZUNG_LE1,FLY_indiciator
0,1,locker bebautes Wohn(misch)gebiet,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,2
1,2,Wohn(misch)gebiet mittlerer Dichte,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
2,3,dichtes Wohn(misch)gebiet,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
3,4,grossvolumiger und solidaerer Wohn(misch)bau,Wohn- u. Mischnutzung (Schwerpunkt Wohnen),Baulandnutzung,1
4,5,Buero- und Verwaltungsviertel,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
5,6,solitaere Handelsstrukturen,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
6,7,Geschaefts- u. Kern- u. Mischgebiete,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
7,8,Mischnutzung wenig dicht,"Geschaefts,- Kern- und Mischnutzung (Schwerpun...",Baulandnutzung,1
8,9,"Industrie prod. Gewerbe, Grosshandel inkl. Lager",Industrie- und Gewerbenutzung,Baulandnutzung,0
9,10,Kultur und Freizeit und Messe,soziale Infrastruktur,Baulandnutzung,1


In [73]:
union_result

,EAST,NORTH,GRID100mID,OBJECTID,BEZ,ZBEZ,ZGEB,BLK,NUTZUNG_CO,NUTZUNG_LE,NUTZUNG_L0,NUTZUNG_L1,FLAECHE,geometry,area
0,-11200.0,336800.0,EPSG31256_GRID_100m_E-11200N336800,26514.0,13,1310,13100,13100011,30.0,GrÃ¼nlandnutzung,Naturraum,Wald,3.157165e+06,"POLYGON ((-11100.000 336900.000, -11100.000 33...",1470.363271
1,-11200.0,336900.0,EPSG31256_GRID_100m_E-11200N336900,26514.0,13,1310,13100,13100011,30.0,GrÃ¼nlandnutzung,Naturraum,Wald,3.157165e+06,"POLYGON ((-11100.000 337000.000, -11100.000 33...",6068.024773
2,-11200.0,337000.0,EPSG31256_GRID_100m_E-11200N337000,26514.0,13,1310,13100,13100011,30.0,GrÃ¼nlandnutzung,Naturraum,Wald,3.157165e+06,"POLYGON ((-11100.000 337000.000, -11145.897 33...",720.724939
3,-11100.0,336700.0,EPSG31256_GRID_100m_E-11100N336700,26514.0,13,1310,13100,13100011,30.0,GrÃ¼nlandnutzung,Naturraum,Wald,3.157165e+06,"POLYGON ((-11000.000 336800.000, -11000.000 33...",1008.240714
4,-11100.0,336800.0,EPSG31256_GRID_100m_E-11100N336800,26514.0,13,1310,13100,13100011,30.0,GrÃ¼nlandnutzung,Naturraum,Wald,3.157165e+06,"POLYGON ((-11100.000 336900.000, -11000.000 33...",9483.780403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177940,18700.0,353500.0,EPSG31256_GRID_100m_E18700N353500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18700.000 353500.000, 18700.000 3536...",10000.000000
177941,18700.0,353600.0,EPSG31256_GRID_100m_E18700N353600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18700.000 353600.000, 18700.000 3537...",10000.000000
177942,18700.0,353700.0,EPSG31256_GRID_100m_E18700N353700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18700.000 353700.000, 18700.000 3538...",10000.000000
177943,18700.0,353800.0,EPSG31256_GRID_100m_E18700N353800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((18700.000 353800.000, 18700.000 3539...",10000.000000


In [76]:
shp_100m_GRID

,EAST,NORTH,GRID100mID,geometry
0,-11200.0,331000.0,EPSG31256_GRID_100m_E-11200N331000,"POLYGON ((-11200.000 331000.000, -11200.000 33..."
1,-11200.0,331100.0,EPSG31256_GRID_100m_E-11200N331100,"POLYGON ((-11200.000 331100.000, -11200.000 33..."
2,-11200.0,331200.0,EPSG31256_GRID_100m_E-11200N331200,"POLYGON ((-11200.000 331200.000, -11200.000 33..."
3,-11200.0,331300.0,EPSG31256_GRID_100m_E-11200N331300,"POLYGON ((-11200.000 331300.000, -11200.000 33..."
4,-11200.0,331400.0,EPSG31256_GRID_100m_E-11200N331400,"POLYGON ((-11200.000 331400.000, -11200.000 33..."
...,...,...,...,...
68995,18700.0,353500.0,EPSG31256_GRID_100m_E18700N353500,"POLYGON ((18700.000 353500.000, 18700.000 3536..."
68996,18700.0,353600.0,EPSG31256_GRID_100m_E18700N353600,"POLYGON ((18700.000 353600.000, 18700.000 3537..."
68997,18700.0,353700.0,EPSG31256_GRID_100m_E18700N353700,"POLYGON ((18700.000 353700.000, 18700.000 3538..."
68998,18700.0,353800.0,EPSG31256_GRID_100m_E18700N353800,"POLYGON ((18700.000 353800.000, 18700.000 3539..."


In [70]:

##Real_land_use2020_100m_test_group_by3

out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m_max_land_use_in_cell.shp"


#
#gdf = shp_100m_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
union_result['area'] = union_result.geometry.area
#union_result
# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
grouped_stats = union_result.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID100mID' and find the index of the row with the maximum area for each group
##idx = gdf.groupby('GRID100mID')['area'].idxmax()




In [2]:

##Real_land_use2020_100m_test_group_by3

#out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m_max_land_use_in_cell.shp"




#
#gdf = shp_100m_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
gdf['area'] = gdf.geometry.area

# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
##grouped_stats = gdf.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID100mID' and find the index of the row with the maximum area for each group
##idx = gdf.groupby('GRID100mID')['area'].idxmax()




In [3]:
## filter by land luse




#import pandas as pd
#selected_rows = gdf[gdf['NUTZUNG_CO'] == 30]
#grouped_stats_30 = selected_rows.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()
#grouped_stats_30['FLY_indiciator'] = 1
#
#grouped_stats_30

GRID100mID                          NUTZUNG_CO
EPSG31256_GRID_100m_E-10000N335000  30.0            370.597432
EPSG31256_GRID_100m_E-10000N335100  30.0           8853.027136
EPSG31256_GRID_100m_E-10000N335200  30.0           9696.248073
EPSG31256_GRID_100m_E-10000N335300  30.0           8776.315390
EPSG31256_GRID_100m_E-10000N335400  30.0          10000.000000
                                                      ...     
EPSG31256_GRID_100m_E9900N344300    30.0           3234.316662
EPSG31256_GRID_100m_E9900N345100    30.0           3516.093626
EPSG31256_GRID_100m_E9900N345200    30.0           3757.677364
EPSG31256_GRID_100m_E9900N345300    30.0           1388.749225
FLY_indiciator                                        1.000000
Name: area, Length: 13433, dtype: float64

In [72]:
grouped_stats

GRID100mID                          NUTZUNG_CO
EPSG31256_GRID_100m_E-10000N335000  30.0           370.597432
EPSG31256_GRID_100m_E-10000N335100  30.0          8853.027136
EPSG31256_GRID_100m_E-10000N335200  30.0          9696.248073
                                    31.0           303.751927
EPSG31256_GRID_100m_E-10000N335300  30.0           8776.31539
                                                     ...     
EPSG31256_GRID_100m_E9900N349000    15.0            63.208506
                                    20.0           448.974406
                                    31.0          1620.726933
FLY_indiciator                                            0.0
NUTZUNG_LE3                                                xx
Name: area, Length: 114059, dtype: object

In [ ]:
## loop by land use table:
union_result


print ("-------------------------- START:")
union_result['FLY_indiciator'] = 0
union_result['NUTZUNG_LE3'] = 'xx'



for i in range(len(df_land_use_LUT['NUTZUNG_CODE'])):
    code = df_land_use_LUT['NUTZUNG_CODE'][i]
    le3 = df_land_use_LUT['NUTZUNG_LE3'][i]
    #le2 = df_land_use_LUT['NUTZUNG_LE2'][i]
    #le1 = df_land_use_LUT['NUTZUNG_LE1'][i]
    fly_indicator = df_land_use_LUT['FLY_indiciator'][i]
    
    print ('Land_use_code   '+ str(code))
    print (le3)
    print ('Fly indicator:   '+ str(fly_indicator))

    union_result.loc[union_result['NUTZUNG_CO'] == code, 'FLY_indiciator'] = fly_indicator
    union_result.loc[union_result['NUTZUNG_CO'] == code, 'NUTZUNG_LE3'] = le3 
    selected_rows = union_result[union_result['NUTZUNG_CO'] == code]



    grouped_stats_lu = selected_rows.groupby(['GRID100mID','NUTZUNG_CO','NUTZUNG_LE3','FLY_indiciator'])['area'].sum()   # ok bis hier
    grouped_stats_lu = grouped_stats_lu.reset_index()
   
  
    
    merged_gdf = shp_100m_GRID.merge(grouped_stats_lu, on='GRID100mID', how='left')

    #print(merged_gdf.head())

 

    out_dem_vector  =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\land_use\Real_land_use2020_100m_max_land_use_in_cell_lucode_"+str(code)+".shp"
    merged_gdf.to_file(out_dem_vector)
    print ("done")


    print ("--------------------------")
    
   
   
          

## 3) Export shp files into 100m raster files

In [32]:
import os
import geopandas as gpd
            
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import numpy as np


## read folder:

folder_path = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\land_use"
multiband_raster = "N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use\\Real_land_use2020_100m_tif"  ## name of the result


raster_list = []


for file in os.listdir(folder_path):
        # Check if the file is a shapefile
        if file.endswith(".shp"):
            # Read the shapefile and append to the list
            gdf = gpd.read_file(os.path.join(folder_path, file))
            print (gdf)

            ## set NULL to 0
            gdf['NUTZUNG_CO'] = gdf['NUTZUNG_CO'].replace(np.nan, 0)
            ## set NULL to 0 AND calc cover of 100x100m cell
            gdf['cover'] = gdf['area']/100  # to %
            gdf['cover'] = gdf['cover'].replace(np.nan, 0)
            gdf["cover"] = gdf["cover"].astype("int")
            
            
            ## raster to vector using vector attriubte (working perfectly):



            # Load the polygon layer path= file_path[:-4]+'.zip'
            polygons =gdf

            output_raster = folder_path+ "\\"+file[:-4]+'.tif'

            
            raster_list.append(output_raster)
          


            # Define the attribute to use for raster values
            attribute_to_rasterize = 'cover'

            # Define raster properties
            #xmin, ymin, xmax, ymax = polygons.total_bounds

            xmin=	-11200
            ymin=	331000
            xmax=	18800
            ymax=	354000

            projection ='epsg:31256'


            pixel_size = 100  # Adjust as needed


            width = int((xmax - xmin) / pixel_size)
            height = int((ymax - ymin) / pixel_size)
            transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

            # Create an empty array to hold rasterized values
            raster_array = rasterize(
                [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
                out_shape=(height, width),
                transform=transform,
                fill=0,
                all_touched=False,  # or True
                dtype=rasterio.uint8
            )

            # Write the raster to a GeoTIFF file
            with rasterio.open(output_raster, "w", driver="GTiff", 
                            height=raster_array.shape[0], 
                            width=raster_array.shape[1], 
                            count=1, 
                            dtype=raster_array.dtype, 
                            crs=polygons.crs, 
                            transform=transform
                            ) as dst:
                dst.write(raster_array, 1)


            print ("done")

##### produce a multiband raster:


file_list = raster_list

# Read metadata of first file
with rasterio.open(file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(file_list))

# Read each layer and write it to stack
with rasterio.open(multiband_raster, 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))


print ("job done")

          EAST     NORTH                          GRID100mID  NUTZUNG_CO  \
0     -11200.0  331000.0  EPSG31256_GRID_100m_E-11200N331000         NaN   
1     -11200.0  331100.0  EPSG31256_GRID_100m_E-11200N331100         NaN   
2     -11200.0  331200.0  EPSG31256_GRID_100m_E-11200N331200         NaN   
3     -11200.0  331300.0  EPSG31256_GRID_100m_E-11200N331300         NaN   
4     -11200.0  331400.0  EPSG31256_GRID_100m_E-11200N331400         NaN   
...        ...       ...                                 ...         ...   
68995  18700.0  353500.0   EPSG31256_GRID_100m_E18700N353500         NaN   
68996  18700.0  353600.0   EPSG31256_GRID_100m_E18700N353600         NaN   
68997  18700.0  353700.0   EPSG31256_GRID_100m_E18700N353700         NaN   
68998  18700.0  353800.0   EPSG31256_GRID_100m_E18700N353800         NaN   
68999  18700.0  353900.0   EPSG31256_GRID_100m_E18700N353900         NaN   

      NUTZUNG_LE FLY_indici  area  \
0           None       None   NaN   
1           N

In [27]:
raster_list

['N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use_test\\Real_land_use2020_100m_max_land_use_in_cell_lucode_1.tif',
 'N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use_test\\Real_land_use2020_100m_max_land_use_in_cell_lucode_10.tif']

In [31]:
import rasterio

multiband_raster = "N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use_test\\Real_land_use2020_100m_tif"

file_list = raster_list

# Read metadata of first file
with rasterio.open(file_list[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(file_list))

# Read each layer and write it to stack
with rasterio.open(multiband_raster, 'w', **meta) as dst:
    for id, layer in enumerate(file_list, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))


print ("job done")

job done


# 4) Alternative: for every 100m  calc. max land use:

In [42]:
##Real_land_use2020_100m_test_group_by3
# 1 Load GRID
shp_100m_GRID = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m.shp")
out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Real_land_use2020_100m_max_land_use_in_cell.shp"

gdf = shp_100m_GRID
#print(gdf.head())


# Calculate the area for each geometry in the GeoDataFrame
gdf['area'] = gdf.geometry.area

# Print the first few rows to check the calculated areas
#print(gdf[['geometry', 'area']].head())


# Group by the attribute field and calculate aggregate statistics
grouped_stats = gdf.groupby(['GRID100mID','NUTZUNG_CO'])['area'].sum()


# Group by 'GRID100mID' and find the index of the row with the maximum area for each group
idx = gdf.groupby('GRID100mID')['area'].idxmax()
# Use the index to select the corresponding 'land_use' value
max_land_use = gdf.loc[idx, ['GRID100mID', 'NUTZUNG_CO']]
max_land_use = max_land_use.rename(columns={'NUTZUNG_CO': 'NUTZ_MAX'})
print(max_land_use.head())
merged_gdf = gdf.merge(max_land_use, on='GRID100mID', how='left')

#print(merged_gdf.head())

# Dissolve the geometries based on the attribute field
dissolved_gdf = merged_gdf.dissolve(by=['GRID100mID','NUTZ_MAX'])



dissolved_gdf.to_file(out_dem_vector)
print ("done")



                                GRID100mID  NUTZ_MAX
151935  EPSG31256_GRID_100m_E-10000N331000       NaN
151936  EPSG31256_GRID_100m_E-10000N331100       NaN
151937  EPSG31256_GRID_100m_E-10000N331200       NaN
151938  EPSG31256_GRID_100m_E-10000N331300       NaN
151939  EPSG31256_GRID_100m_E-10000N331400       NaN
done


# 5) Fly indicator based on the 100m raster collecction

In [53]:

import os
import geopandas as gpd
            
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize
from rasterio.transform import from_bounds

import numpy as np



multiband_raster = "N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use\\Real_land_use2020_100m_tif"  ## multiband raster
land_use_fly_indicator = "N:\\C2205_FAIRiCUBE\\f02_data\\d050_data_WIEN\\Vienna_gridded_data_collection\\100m_GRID\\land_use\\land_use_fly_indicator_100m_tif"  ## multiband raster


dataset = rasterio.open(multiband_raster)
print ("count of bands:")
count_of_bands =dataset.count
print (count_of_bands)


# read the single land use classes:
band1 = dataset.read(1)
band2 = dataset.read(2)
band3  = dataset.read(3)
band4  = dataset.read(4)
band5  = dataset.read(5)
band6  = dataset.read(6)
band7  = dataset.read(7)
band8  = dataset.read(8)
band10  = dataset.read(10)
band11  = dataset.read(11)
band12  = dataset.read(12)
band14  = dataset.read(14)
band15  = dataset.read(15)
band19  = dataset.read(19)
band24  = dataset.read(24)
band25  = dataset.read(25)
band26  = dataset.read(26)
band27  = dataset.read(27)
band28  = dataset.read(28)
band29  = dataset.read(29)
band30  = dataset.read(30)
band31  = dataset.read(31)

# calc indicator
land_use_fly_raster = band1 *2 +band2 +band3 +band4 +band5 +band6 +band7 +band8 +band10+band11+band12+band14+band15 *2 +band19+band24+band25+band26+band27+band28*2+band29*2+band30+band31
print("value of max and mean", land_use_fly_raster.max(), land_use_fly_raster.min()) 


# normalize the data
land_use_fly_raster_norm = (land_use_fly_raster-np.min(land_use_fly_raster))/(np.max(land_use_fly_raster)-np.min(land_use_fly_raster))
print("value of max and mean", land_use_fly_raster_norm.max(), land_use_fly_raster_norm.min()) 


with rasterio.open(multiband_raster) as src:
    # Read the raster values
    raster_values = src.read(1)  # Change the band index if necessary


    # Define the transformation parameters based on the dimensions of the raster and pixel size
    # Adjust these parameters according to your raster dimensions and pixel size
    transform = from_origin(0, 0, pixel_size, pixel_size)

    # Create a raster profile
    profile = src.profile

    # Write the normalized values to the raster file
    with rasterio.open(land_use_fly_indicator, 'w', **profile) as dst:
        dst.write(land_use_fly_raster_norm *100 , 1)  # Write the data to the first band

print("Raster file written successfully:", land_use_fly_indicator)



print ("job done")

count of bands:
32
value of max and mean 200 0
value of max and mean 1.0 0.0
Raster file written successfully: N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\land_use\land_use_fly_indicator_100m_tif
job done
